# Chat with BDB


### Sources
Video: https://www.youtube.com/watch?v=3yPBVii7Ct0
Adapted from: https://colab.research.google.com/drive/1gyGZn_LZNrYXYXa-pltFExbptIe7DAPe?usp=sharing#scrollTo=XHVE9uFb3Ajj

### Contents
1. Setting up
2. Load txt documents
3.

## 1. Setting up


In [1]:
!pip show langchain

Name: langchain
Version: 0.0.136
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/michielbontenbal/anaconda3/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [13]:
import os
import config
os.environ["OPENAI_API_KEY"] = config.openai_key

In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

## 2. Load multiple .pdf documents, convert to text and split into chunks

In [15]:
pwd

'/Users/michielbontenbal/Documents/GitHub/chat_with_documents/ChatBDB'

In [1]:
!find . -type f -name "*.pdf"

./Wijze_Lessen_digitaal_160919.pdf
./Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023 (1).pdf
./Strategie 4 Activeren van studenten als belangrijke informatiebronnen voor elkaar.pdf
./Hoge verwachtingen voor elke leerling_ je sterkste wapen tegen ongelijkheid – Klasse.pdf
./Handout BDB Bijeenkomst 7 - Groep W2 Erwin - 5 april 2023.pdf
./Handout Wijze lessen 12 bouwstenen voor effectieve didactiek1.pdf
./HvA-infographic-onderwijs-toetsbeleid.pdf
./Five_Key_Strategies_for_Effective_Formative_Assessment.pdf
./Sluijsmans (2020).pdf
./Didactisch Coachen. Lia Voerman en Frans Faber.pdf
./Strategie 1 Verhelderen en begrijpen van leerdoelen.pdf
./Strategie 2 Realiseren van effectieve taken, discussies en activiteiten die het leren zichtbaar maken.pdf
./Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023.pdf
./Handout BDB Bijeenkomst 8 - Groep W2 Erwin - 12 april  2023.pdf
./Strategie 5 Activeren van studenten in het stimuleren van eigenaar

In [3]:
from glob import glob

my_pdfs = glob('*.pdf')
my_pdfs

['Wijze_Lessen_digitaal_160919.pdf',
 'Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023 (1).pdf',
 'Strategie 4 Activeren van studenten als belangrijke informatiebronnen voor elkaar.pdf',
 'Hoge verwachtingen voor elke leerling_ je sterkste wapen tegen ongelijkheid – Klasse.pdf',
 'Handout BDB Bijeenkomst 7 - Groep W2 Erwin - 5 april 2023.pdf',
 'Handout Wijze lessen 12 bouwstenen voor effectieve didactiek1.pdf',
 'HvA-infographic-onderwijs-toetsbeleid.pdf',
 'Five_Key_Strategies_for_Effective_Formative_Assessment.pdf',
 'Sluijsmans (2020).pdf',
 'Didactisch Coachen. Lia Voerman en Frans Faber.pdf',
 'Strategie 1 Verhelderen en begrijpen van leerdoelen.pdf',
 'Strategie 2 Realiseren van effectieve taken, discussies en activiteiten die het leren zichtbaar maken.pdf',
 'Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023.pdf',
 'Handout BDB Bijeenkomst 8 - Groep W2 Erwin - 12 april  2023.pdf',
 'Strategie 5 Activeren van studenten in

In [9]:
#THE one script to make one long txt file (by adding all pages to the text file)
from PyPDF2 import PdfReader
from glob import glob
import os

my_pdfs = glob('*.pdf')

for i in range(len(my_pdfs)):
    reader = PdfReader(my_pdfs[i])
    number_of_pages = len(reader.pages)
    file_name, file_extension = os.path.splitext(my_pdfs[i])
    textfile = open(file_name+".txt", "w")

    for j in range (number_of_pages):
        page = reader.pages[j]
        textfile.write(page.extract_text())
        textfile.write('}\n')
    textfile.close() 

In [16]:
!find . -type f -name "*.txt"

./Handout Wijze lessen 12 bouwstenen voor effectieve didactiek1.txt
./HvA-infographic-onderwijs-toetsbeleid.txt
./Handout BDB Bijeenkomst 7 - Groep W2 Erwin - 5 april 2023.txt
./Hoge verwachtingen voor elke leerling_ je sterkste wapen tegen ongelijkheid – Klasse.txt
./Five_Key_Strategies_for_Effective_Formative_Assessment.txt
./Sluijsmans (2020).txt
./Didactisch Coachen. Lia Voerman en Frans Faber.txt
./Wijze_Lessen_digitaal_160919.txt
./Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023 (1).txt
./Strategie 4 Activeren van studenten als belangrijke informatiebronnen voor elkaar.txt
./Strategie 5 Activeren van studenten in het stimuleren van eigenaarschap.txt
./Strategie 3 Feedback en feedforward geven gericht op verder leren.txt
./Strategie 2 Realiseren van effectieve taken, discussies en activiteiten die het leren zichtbaar maken.txt
./Handout BDB bijeenkomst 9 Onderwijsontwerp - Groep W2 (Erwin) - 19 april 2023.txt
./Strategie 1 Verhelderen en begrijpen van

In [17]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [18]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [19]:
len(texts)

605

In [20]:
texts[3]

Document(page_content='voorbeelden. Bespreek waar \n(in de opleiding)  \nen hoe je aan de \nslag wilt.Doe een pilot en \nbespreek je bevindingen \nmet collega’s en andere \nbetrokkenen.Hoe ga je ermee aan de slag?\nOnderwijs- en toetsbeleid voor je opleiding maak je niet alleen.  \nBreng in kaart wie je gaat betrekken van jouw opleiding en denk \ndaarbij ook aan studenten en werkveld als partners.\nScan de code voor de Sharepoint\nvan onderwijs- en toetsbeleid.\nBespreek  \nwat bij jouw \nopleiding past.Maak een afspraak om te \nsparren met een collega die al \nervaring heeft opgedaan met \neen specifiek onderwerp.Hier vind je de leden\nvan de werkgroep OTBGebruik de \ninformatie op \nSharepoint. Wat \nheb je nog nodig?\nMaak een plan.\nOrganiseer  \neen werksessie.ORIËNTEREN\n& INSPIRERENTOEPASSEN}', metadata={'source': 'HvA-infographic-onderwijs-toetsbeleid.txt'})

## 3. Create the vector database

In [21]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

Using embedded DuckDB with persistence: data will be stored in: db


In [22]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [23]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

Using embedded DuckDB with persistence: data will be stored in: db


## 4. Make a retriever

In [24]:
retriever = vectordb.as_retriever()

In [25]:
docs = retriever.get_relevant_documents("Wat is didactisch coachen?")

In [26]:
len(docs)

4

In [27]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [28]:
retriever.search_type

'similarity'

In [29]:
retriever.search_kwargs

{'k': 2}

## 5. Make a chain

In [30]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [31]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [32]:
# full example
query = "What is didactisch coachen?"
llm_response = qa_chain(query)
#process_llm_response(llm_response)
llm_response

{'query': 'What is didactisch coachen?',
 'result': ' Didactisch Coachen is een planmatige en doelgericht e wijze van coachen met behulp van een aantal procesgerichte interventies die het leren en de motivatie van de lerende bevordert. Didactisch Coachen beoogt leerlingen actief en gemotiveerd te maken en te houden door het stellen van denkstimulerende vragen die bevorderlijk zijn voor het leerproces en feedback mogelijk maken, het geven van effectieve feedback die het leren positief beïnvloedt, het beperkt geven van aanwijzingen, en dit alles op zowel de dimensies van de taak (inhoud, strategie), als op de dimensies van de lerende persoon (modus / leerstand en de persoonlijke kwaliteiten).',
 'source_documents': [Document(page_content='Didactisch Coachen is een planmatige en doelgericht e wijze van coachen met behulp \nvan een aantal procesgerichte interventies die het leren en de motivatie van de lerende \nbevordert. Didactisch Coachen beoogt leerlingen actief en gemotiveerd te maken

In [34]:
# break it down
query = "Wat zijn de bouwstenen uit wijze lessen?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'Wat zijn de bouwstenen uit wijze lessen?',
 'result': ' De bouwstenen uit Wijze Lessen zijn twaalf bouwstenen voor effectief lesgeven.',
 'source_documents': [Document(page_content='27}\nWijze Lessen28 | \nHoe gebruik je dit boek?\nDit boek gaat uitdrukkelijk niet over het maken van een lesvoorbereiding, \nover klasmanagement, over de constructie van de perfecte toets, over administratie, over leer- en motivatietheorieën. Het boek gaat wel over de kern van ons vak: het lesgeven.\nJe maakt in dit boek kennis met twaalf bouwstenen voor effectief lesge -\nven. Je kunt als leraar tegen dit boek op verschillende manieren aankijken.', metadata={'source': 'Wijze_Lessen_digitaal_160919.txt'}),
  Document(page_content="WIJZE LESSEN\nTwaalf bouwstenen voor eﬀectieve didactiek\nTim Surma • Kristel Vanhoyweghen • Dominique Sluijsmans\nGino Camp • Daniel Muijs • Paul A. Kirschner}\n‘Onderwijs is het punt waarop we beslissen \nof we de wereld genoeg lief hebben om er \nverantwoordelijkhei

In [42]:
# break it down
query = "Wat is cognitieve belasting?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'Wat is cognitieve belasting?',
 'result': " Cognitieve belasting is een theorie ontwikkeld door John Sweller en collega's in de jaren 80. Het betekent het beheren van de belasting van het werkgeheugen zodat de extrinsieke belasting geminimaliseerd wordt en de cognitieve capaciteit optimaal gebruikt wordt binnen de grenzen van de beschikbare bandbreedte van de leerling.",
 'source_documents': [Document(page_content='Optimale belasting van het werkgeheugen: \nniet te veel, niet te weinig. \nDeze kenmerken van ons brein hebben belangrijke didactische gevol-\ngen. Zeker als het werkgeheugen van leerlingen nieuwe informatie te verwerken krijgt in een les, is het belangrijk om de beperkingen van dat werkgeheugen te kennen. Dit idee vormde de basis voor de cognitieve-belastingtheorie (cognitive load theory). Deze theorie is de jaren tachtig ontwikkeld door John Sweller en collega’s.\n11\nStel dat je als lerende een maximale capaciteit van je werkgeheugen hebt, noem het ‘mentale ban

In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f9f7dc82aa0>)

In [25]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
